In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchtext

from transformers import BertTokenizer, BertForSequenceClassification , BertModel , AutoTokenizer, BertForSequenceClassification


import math
import time

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()

        # 日本語学習済モデルをロードする
        # output_attentions=Trueで順伝播のときにattention weightを受け取れるようにする
        # output_hidden_state=Trueで12層のBertLayerの隠れ層を取得する
        self.bert = model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', return_dict=True)

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元、出力はポジ・ネガの2つ
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    # clsトークンのベクトルを取得する用の関数を用意
    def _get_cls_vec(self, vec):
        return vec[:,0,:].view(-1, 768)

    def forward(self, input_ids):

        # 順伝播の出力結果は辞書形式なので、必要な値のkeyを指定して取得する
        output = self.bert(input_ids)
        #attentions = output['attentions']
        hidden_states = output['hidden_states']

        # 最終４層の隠れ層からそれぞれclsトークンのベクトルを取得する
        vec1 = self._get_cls_vec(hidden_states[-1])
        vec2 = self._get_cls_vec(hidden_states[-2])
        vec3 = self._get_cls_vec(hidden_states[-3])
        vec4 = self._get_cls_vec(hidden_states[-4])

        # 4つのclsトークンを結合して１つのベクトルにする。
        vec = torch.cat([vec1, vec2, vec3, vec4], dim=1)

        # 全結合層でクラス分類用に次元を変換
        out = self.linear(vec)
        

        return F.log_softmax(out, dim=1)#, attentions


# モデル構築
classifier = BertClassifier()
classifier = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', return_dict=True)
# 訓練モードに設定
classifier.train()

print('ネットワーク設定完了')


In [ ]:
from transformers import BertModel
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# BERTはサブワードを含めて最大512単語まで扱える
MAX_LENGTH = 512
def bert_tokenizer(text):
    return tokenizer.encode(text, max_length=MAX_LENGTH, truncation=True, return_tensors='pt')[0]

フィールドの設定
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=bert_tokenizer, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=MAX_LENGTH, pad_token=0)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

train_data, test_data = torchtext.legacy.data.TabularDataset.splits(
    path='./', train='train.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT),('Label', LABEL)])

#BERTではミニバッチサイズは16か32が推奨される
BATCH_SIZE = 8
train_iter, test_iter = torchtext.legacy.data.Iterator.splits((train_data, test_data), batch_sizes=(BATCH_SIZE, BATCH_SIZE), repeat=False, sort=False)

dataloaders_dict = {"train": train_iter, "val": test_iter}



In [ ]:
# ミニバッチの1文目を確認してみる
text_minibatch_1 = (batch.Text[0][1]).numpy()

# IDを単語に戻す
text = tokenizer.convert_ids_to_tokens(text_minibatch_1)

print(text)
a = tokenizer.encode_plus(text, return_tensors='pt', padding=True, truncation=True)
#len(a['attention_mask']),len(a['input_ids']),type(a['attention_mask'])
print(a)
len(batch.Text[0][1])

In [ ]:
# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for batch in train_iter:# train_dataloaderはword_id, mask, labelを出力する点に注意
        attention_mask = [1] * len(batch.Text[0][0])
        print(attention_mask)
        input_ids = batch.Text[0].to(device)
        labels = batch.Label[0].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, token_type_ids = None,attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #log_str = "loss値は「%s」" % str(loss.item())
        #print(log_str)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model.eval()# 検証モードに切り替え
  val_loss = 0
  with torch.no_grad(): # 訓練じゃないので勾配を計算しない
    for batch in test_iter:
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      with torch.no_grad():
        (loss, logits) = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask,
                              labels=b_labels)
        val_loss += loss.item()
    return val_loss

In [ ]:
# 学習の実行
torch.cuda.empty_cache()

classifier = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', return_dict=True)

from transformers import AdamW
optimizer = AdamW(classifier.parameters(), lr=1e-5) # AdamWオプティマイザ

max_epoch = 20
train_loss_ = []
test_loss_ = []

#エポック数の設定
for epoch in range(max_epoch):
    train_ = train(classifier)
    train_loss_.append(train_)

In [ ]:
train = pd.read_csv('train.tsv',sep = '\t',header = None)
test = pd.read_csv('test.tsv',sep = '\t',header = None)
train

In [ ]:
text = list(train[0])
label = list(train[1])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from sklearn.model_selection import train_test_split

#tensor型に変換
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
label = torch.tensor(label)

# データセットクラスの作成
# なんでTensorDataset使ってデータを分割しているかって？
# 分割しなくても良いけど、その場合、メモリが16G程度だとすぐにRAMがいっぱいになてしまうんだ
dataset = TensorDataset(input_ids, attention_mask, label)


#トレーニングデータ、評価データを9:1の割合で分割する
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを訓練用と検証用に分ける
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(train_dataset[0])
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100,
#                                                    stratify=y)

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 1

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', return_dict=True)

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5) # AdamWオプティマイザ

In [ ]:
# 訓練パートの定義
def train(model,train_dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.train() # 訓練モードで実行
    model.to(device)
    train_loss = 0


    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        #outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #log_str = "loss値は「%s」" % str(loss.item())
        #print(log_str)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 検証モードに切り替え
    val_loss = 0
    with torch.no_grad(): # 訓練じゃないので勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            val_loss += loss.item()
    return val_loss


In [ ]:
# 学習の実行
max_epoch = 20
train_loss_ = []
test_loss_ = []

#エポック数の設定
for epoch in range(max_epoch):
    train_ = train(model,train_dataloader)
    train_loss_.append(train_)

    print(train_)